In [1]:
import hail as hl
hl.init() 

Running on Apache Spark version 2.4.1
SparkUI available at http://npept149a.if-pan.krakow.pl:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.26-2dcc3d963867
LOGGING: writing to /home/ifpan/projects/imdik-zekanowski-gts/hail-20191112-1447-0.2.26-2dcc3d963867.log


In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [10]:
hl.import_vcf('data/annotated.test.vcf', reference_genome='GRCh38').write('data/sample.mt', overwrite=True)

2019-11-12 15:44:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-11-12 15:45:18 Hail: INFO: wrote matrix table with 726380 rows and 151 columns in 116 partitions to data/sample.mt


In [11]:
mt = hl.read_matrix_table('data/sample.mt')

We can use rows along with select to pull out 5 variants. The select method takes either a string refering to a field name in the table, or a Hail Expression. Here, we leave the arguments blank to keep only the row key fields, locus and alleles.

In [12]:
mt.rows().select().show(5)

locus,alleles
locus<GRCh38>,array<str>
chr1:10146,"[""AC"",""*""]"
chr1:10177,"[""A"",""*""]"
chr1:10622,"[""T"",""*""]"
chr1:10623,"[""T"",""*""]"
chr1:30923,"[""G"",""*""]"


In [18]:
mt.entry.take(5) #create a list

[Struct(AD=[2, 5, 0], DP=7, GQ=32, GT=Call(alleles=[0, 0], phased=False), PGT=None, PID=None, PL=[114, 120, 168]),
 Struct(AD=[42, 0, 0], DP=42, GQ=None, GT=None, PGT=None, PID=None, PL=None),
 Struct(AD=[65, 0, 0], DP=65, GQ=None, GT=None, PGT=None, PID=None, PL=None),
 Struct(AD=[32, 0, 0], DP=32, GQ=None, GT=None, PGT=None, PID=None, PL=None),
 Struct(AD=[42, 0, 0], DP=42, GQ=None, GT=None, PGT=None, PID=None, PL=None)]

In [15]:
mt.s.show(5) #samples


s
str
"""S_136"""
"""S_170c"""
"""S_170d"""
"""S_6981"""
"""S_6982"""


In [19]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DB: bool, 
        DP: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        MULTIALLELIC: bool, 
        ALTS: array<str>, 
        ALTS_pos: int32, 
        ALTS_indices: array<str>, 
        ISEQ_GNOMAD_EXOMES_AC: array<int32>, 
        ISEQ_GNOMAD_EXOMES_AN: int32, 
        ISEQ_GNOM